<a href="https://colab.research.google.com/github/cbun1123/map_fit/blob/main/Stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/cbun1123/map_fit

!unzip /content/map_fit/X_test_1.zip -d /content/map_fit/X_test
!unzip /content/map_fit/X_test_2.zip -d /content/map_fit/X_test
!unzip /content/map_fit/X_test_3.zip -d /content/map_fit/X_test

In [ ]:
!nvidia-smi
!pip install pytorch-lightning
%env PYTHONPATH=.:$PYTHONPATH

In [ ]:
import numpy as np
import cv2
import glob
import math

from google.colab.patches import cv2_imshow

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

import torchvision
from torchvision import transforms

In [ ]:
filenames = glob.glob("/content/map_fit/X_test/*.jpg")
filenames.sort()
images = [cv2.imread(img) for img in filenames]

X_train = np.stack(images, axis=0)
y_train = np.load('/content/map_fit/Y_test.npy')

SIZE = 224

lin = np.linspace(-1,1,SIZE)
[Xm,Ym] = np.meshgrid(lin,lin)
idx = ((Xm**2+Ym**2)<1)
idx = np.stack([idx,idx,idx], axis=2)
idx_t = torch.from_numpy(idx)

class imageDataset(Dataset):
    def __init__(self):
        'Initialization'
        self.X = X_train
        self.y = y_train
        self.n_samples = self.X.shape[0]
        self.transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        image = self.X[index]
        X = self.transform(image)
        return X

    def __len__(self):
        return self.n_samples


dataset = imageDataset()
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
num_epochs = 2
total_samples = len(dataset)
n_iteration = math.ceil(total_samples/32)

for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(dataloader):
        # Trucs
        print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iteration}, inputs {inputs.shape}')

ValueError: ignored